# Spherical Gaussian experiment: generative results

## Setup

In [1]:
import sys
import numpy as np
import torch
from torch.utils.data import TensorDataset
import logging
from sklearn.metrics import roc_auc_score

sys.path.append("../../")

logging.basicConfig(
    format="%(asctime)-5.5s %(name)-30.30s %(levelname)-7.7s %(message)s",
    datefmt="%H:%M",
    level=logging.DEBUG,
)
logger = logging.getLogger(__name__)
# Output of all other modules (e.g. matplotlib)
for key in logging.Logger.manager.loggerDict:
    if "manifold_flow" not in key:
        logging.getLogger(key).setLevel(logging.WARNING)


## Load results

In [16]:
setup_filenames = [
    "2_3_0.010",
    "2_3_0.001",
    "2_3_0.100",
]
setup_labels = [
    "epsilon = 0.01",
    "epsilon = 0.001",
    "epsilon = 0.1",
]

In [3]:
algo_filenames = []
algo_additionals = []
algo_labels = []

def add_algo(filename, add, label):
    algo_filenames.append(filename)
    algo_additionals.append(add)
    algo_labels.append(label)
    
    
add_algo("flow", "_small", "Flow")
add_algo("flow", "_small_long", "Flow (long)")
add_algo("flow", "_small_shallow_long", "Flow (shallow, long)")

add_algo("pie", "_small", "PIE") 
add_algo("pie", "_small_long", "PIE (long)") 
add_algo("pie", "_small_shallow_long", "PIE (shallow, long)") 

add_algo("mf", "_small", "MAD AF")
add_algo("mf", "_small_noprepost", "MAD AF (no pre/post)")
add_algo("mf", "_small_complex", "MAD AF (complex)") 
add_algo("mf", "_small_long", "MAD AF (long)")
add_algo("mf", "_small_shallow_long", "MAD AF (shallow, long)")

add_algo("emf", "_small", "MAD AF + Enc.")

add_algo("gamf", "_small_largebs", "OT MAD AF (Sinkhorn)")
add_algo("gamf", "_small_ged_largebs", "OT MAD AF (GED)") 
add_algo("gamf", "_small_ged_largebs_long", "OT MAD AF (GED, long)") 
add_algo("gamf", "_small_ged_largebs_shallow_long", "OT MAD AF (GED, shallow, long)") 

add_algo("pie_specified", "_small", "Prescr. PIE") 
add_algo("pie_specified", "_small_long", "Prescr. PIE (long)") 
add_algo("pie_specified", "_small_shallow_long", "Prescr. PIE (shallow, long)") 

add_algo("mf_specified", "_small", "Prescr. MAD AF") 
add_algo("mf_specified", "_small_long", "Prescr. MAD AF (long)") 
add_algo("mf_specified", "_small_shallow_long", "Prescr. MAD AF (shallow, long)")

add_algo("gamf_specified", "_small_largebs", "Prescr. OT MAD AF (Sinkhorn)")
add_algo("gamf_specified", "_small_ged_largebs", "Prescr. OT MAD AF (GED)") 
add_algo("gamf_specified", "_small_ged_largebs_long", "Prescr. OT MAD AF (GED, long)") 
add_algo("gamf_specified", "_small_ged_largebs_shallow_long", "Prescr. OT MAD AF (GED, shallow, long)") 


In [4]:
def load(name, shape, numpyfy=True, result_dir="../data/results"):
    all_results = []
    
    for algo_filename, algo_add in zip(algo_filenames, algo_additionals):
        results = []
        
        for setup_filename in setup_filenames:
            try:
                results.append(np.load(
                    "{}/{}_2_spherical_gaussian_{}{}_{}.npy".format(
                        result_dir, algo_filename, setup_filename, algo_add, name
                    )
                ))
            except FileNotFoundError as e:
                # print(e)
                
                if shape is None:
                    results.append(None)
                else:
                    results.append(np.nan*np.ones(shape))
            
        all_results.append(results)
    
    return np.asarray(all_results) if numpyfy else all_results

x_gen = load("samples", None, numpyfy=False)
logp_gen = load("samples_likelihood", (10000,))
distance_gen = load("samples_manifold_distance", (10000,))

true_test_log_likelihood = load("true_log_likelihood_test", (1000,))
model_test_log_likelihood = load("model_log_likelihood_test", (1000,))
model_test_reco_error = load("model_reco_error_test", (1000,))

model_ood_log_likelihood = load("model_log_likelihood_ood", (1000,))
model_ood_reco_error = load("model_reco_error_ood", (1000,))


In [5]:
x_test = np.asarray([
    np.load("../data/samples/spherical_gaussian/spherical_gaussian_{}_x_test.npy".format(setup_filename))
    for setup_filename in setup_filenames
])

true_distances = np.abs(np.sum(x_test**2, axis=-1)**0.5 - 1.)

## Calculate metrics

In [6]:
min_logp = -100.
max_distance = 10.

mean_logp_gen = np.mean(np.clip(logp_gen, min_logp, None), axis=2)
mean_distance_gen = np.mean(np.clip(distance_gen, None, max_distance), axis=2)

mean_logp_truth = np.mean(np.clip(true_test_log_likelihood[0], min_logp, None), axis=1)
mean_distance_truth = np.mean(np.clip(true_distances, None, max_distance), axis=1)

In [7]:
def calculate_roc_auc(x0, x1):
    return np.asarray(
        [
            [
                np.nan if not np.all(np.isfinite(np.hstack((this_x0, this_x1)))) else (
                    np.maximum(
                        roc_auc_score(
                            np.hstack((np.zeros(this_x0.shape[0], dtype=np.int),
                                       np.ones(this_x1.shape[0], dtype=np.int))),
                            np.hstack((this_x0, this_x1)),
                        ),
                        roc_auc_score(
                            np.hstack((np.zeros(this_x0.shape[0], dtype=np.int),
                                       np.ones(this_x1.shape[0], dtype=np.int))),
                            - np.hstack((this_x0, this_x1)),
                        )
                    )
                )
                for this_x0, this_x1 in zip(x0_, x1_)
            ]
            for x0_, x1_ in zip(x0, x1)
        ]
    )


auc_logp = calculate_roc_auc(model_test_log_likelihood, model_ood_log_likelihood)
auc_err = calculate_roc_auc(model_test_reco_error, model_ood_reco_error)
auc_use_err = (auc_err > auc_logp)
auc_best = np.maximum(auc_err, auc_logp)

/Users/johannbrehmer/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in greater


## Print metrics

In [8]:
def print_results(setup):
    print("{:<36.36s} | {:>8.8s} {:>8.8s} {:>8.8s}".format(setup_labels[setup], "Log p", "Dist", "AUC"))
    print("-"*65)
    for label, logp, dist, auc in zip(
        ["Simulator"] + algo_labels,
        [mean_logp_truth] + list(mean_logp_gen),
        [mean_distance_truth] + list(mean_distance_gen),
        [[0.0, 0.0, 0.0]] + list(auc_best)
    ):
        if np.isfinite(logp[setup]) or np.isfinite(dist[setup]):
            print("{:<36.36s} | {:>8.2f} {:>8.3f} {:>8.3f}".format(label, logp[setup], dist[setup], auc[setup]))
        else:
            print("{:<36.36s} |                           ".format(label))

In [18]:
def print_all_results():
    print(
        "{:<36.36s} | {:^20.20s} | {:^20.20s} | {:^20.20s}".format(
        "", setup_labels[0], setup_labels[1], setup_labels[2]
        )
    )
    print(
        "{:<36.36s} | {:>6.6s} {:>6.6s} {:>6.6s} | {:>6.6s} {:>6.6s} {:>6.6s} | {:>6.6s} {:>6.6s} {:>6.6s}".format(
        "", "Log p", "Dist", "AUC", "Log p", "Dist", "AUC", "Log p", "Dist", "AUC"
        )
    )
    print("-"*105)
    for label, logp, dist, auc in zip(
        ["Simulator"] + algo_labels,
        [mean_logp_truth] + list(mean_logp_gen),
        [mean_distance_truth] + list(mean_distance_gen),
        [[0.0, 0.0, 0.0]] + list(auc_best)
    ):
        def _f(val):
            return "{:>6.2f}".format(val) if np.isfinite(val) else "      "
        
        print(
            "{:<36.36s} | {} {} {} | {} {} {} | {} {} {}".format(
                label, _f(logp[0]), _f(dist[0]),  _f(auc[0]), _f(logp[1]), _f(dist[1]),  _f(auc[1]), _f(logp[2]), _f(dist[2]),  _f(auc[2]), 
            )
        )
        

In [19]:
print_all_results()

                                     |    epsilon = 0.01    |   epsilon = 0.001    |    epsilon = 0.1    
                                     |  Log p   Dist    AUC |  Log p   Dist    AUC |  Log p   Dist    AUC
---------------------------------------------------------------------------------------------------------
Simulator                            |   2.32   0.01   0.00 |   4.68   0.00   0.00 |   0.05   0.08   0.00
Flow                                 |   1.31   0.03   0.69 |   2.98   0.03   0.71 |  -0.82   0.10   0.65
Flow (long)                          |   0.40   0.04   0.59 | -43.37   0.04   0.51 |  -0.66   0.10   0.64
Flow (shallow, long)                 |   0.07   0.04   0.58 | -42.11   0.03   0.51 |  -0.89   0.10   0.64
PIE                                  |   1.92   0.02   0.71 |   0.36   0.03   0.67 |  -0.28   0.10   0.65
PIE (long)                           |   0.06   0.04   0.53 | -66.20   0.04   0.52 |  -2.40   0.17   0.64
PIE (shallow, long)                  |        